In [ ]:
import pandas as pd
import glob
import os
import pymysql

# path details of all csv files
folder_path = r'C:\Users\HP\Desktop\kani\project_redbus\Red_bus_final_details'

# Use glob to find all csv files in the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

dfs=[]
# Read each csv file and print or store the content
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)
    
merged_df = pd.concat(dfs, ignore_index=True)   

merged_df.to_csv("all_state_bus_detail.csv", index=False)

### data cleaning and data type changes:
df = pd.read_csv("all_state_bus_detail.csv")


df['Price'] = df['Price'].str.replace('INR', "", regex= False)
df['Price'] = pd.to_numeric(df['Price'], errors = 'coerce')

df.rename(columns={'Seats Available': 'Seat_Availability'}, inplace=True)
df['Seat_Availability'] = df['Seat_Availability'].str.extract(r'(\d+)')
df['Seat_Availability'] = pd.to_numeric(df['Seat_Availability'], errors = 'coerce')

df['Seat_Availability'] = df['Seat_Availability'].astype(int)
df['Route Name'] = df['Route Name'].astype(str)
df['Route Link'] = df['Route Link'].astype(str)
df['Bus Name'] = df['Bus Name'].astype(str)
df['Bus Type'] = df['Bus Type'].astype(str)
df['Departure Time'] = df['Departure Time'].astype(str)
df['Duration'] = df['Duration'].astype(str)
df['Reaching Time'] = df['Reaching Time'].astype(str)

df= df.dropna()
# Connect to MySQL
try:
    myconnection = pymysql.connect(
        host='127.0.0.1',
        user='root',
        passwd='********',
        database='Red_Bus'
    )
    cursor = myconnection.cursor()
    print("Connection successful!")
except pymysql.OperationalError as e:
    print(f"OperationalError: {e}")

# SQL query to create the table
create_table_query = """
CREATE TABLE IF NOT EXISTS Bus_routes (
    ID INT AUTO_INCREMENT PRIMARY KEY,
    Route_Name TEXT,
    Route_Link TEXT,
    Bus_Name TEXT,
    Bus_Type TEXT,
    Departing_Time TIME,
    Duration TEXT,
    Reaching_Time TIME,
    Star_Rating FLOAT,
    Price DECIMAL(10, 2),
    Seat_Availability INT
);
"""

cursor.execute(create_table_query)

print("Table Bus_routes created successfully!")


# Insert data into table
insert_query = """
INSERT INTO Bus_routes (Route_Name, Route_Link, Bus_Name, Bus_Type, Departing_Time, Duration, Reaching_Time, Star_Rating, Price, Seat_Availability)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

data_tuples = [
    (
        row['Route Name'],
        row['Route Link'],
        row['Bus Name'],
        row['Bus Type'],
        row['Departure Time'],
        row['Duration'],
        row['Reaching Time'],
        row['Rating'],
        row['Price'],
        row['Seat_Availability']
    )
    for index, row in df.iterrows()
]

# Use executemany for batch inserts
cursor.executemany(insert_query, data_tuples)
myconnection.commit()
print("Data inserted successfully!")

cursor.close()
myconnection.close()